In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import keras 
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf

In [2]:
labels = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
img_size = 224
def get_data(data_dir):
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img))[...,::-1] #convert BGR to RGB format
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)

In [3]:
train = get_data('dataset')

C:\Users\chandru\AppData\Local\Temp/ipykernel_21460/3359061214.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(data)


In [4]:
x_train = []
y_train = []

for feature, label in train:
  x_train.append(feature)
  y_train.append(label)
    

In [5]:
# Normalize the data
x_train = np.array(x_train) / 255

In [6]:
## reshape

x_train.reshape(-1, img_size, img_size, 1)
y_train = np.array(y_train)

In [7]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train)

In [8]:
model = Sequential()
model.add(Conv2D(32,3,padding="same", activation="relu", input_shape=(224,224,3)))
model.add(MaxPool2D())

model.add(Conv2D(32, 3, padding="same", activation="relu"))
model.add(MaxPool2D())

model.add(Conv2D(64, 3, padding="same", activation="relu"))
model.add(MaxPool2D())
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dense(10, activation="softmax"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 56, 56, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 28, 28, 64)       0

In [9]:
opt = Adam(lr=0.000001)
model.compile(optimizer = opt , loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) , metrics = ['accuracy'])

C:\Users\chandru\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [10]:
history = model.fit(x_train,y_train,epochs = 500)

Epoch 1/500


C:\Users\chandru\anaconda3\lib\site-packages\tensorflow\python\util\dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


5/5 [==============================] - 9s 1s/step - loss: 2.3246 - accuracy: 0.0942
Epoch 2/500
5/5 [==============================] - 5s 1s/step - loss: 2.3106 - accuracy: 0.0942
Epoch 3/500
5/5 [==============================] - 5s 1s/step - loss: 2.3155 - accuracy: 0.0870
Epoch 4/500
5/5 [==============================] - 5s 1s/step - loss: 2.3149 - accuracy: 0.1159
Epoch 5/500
5/5 [==============================] - 5s 994ms/step - loss: 2.2988 - accuracy: 0.1014
Epoch 6/500
5/5 [==============================] - 5s 1s/step - loss: 2.2946 - accuracy: 0.1304
Epoch 7/500
5/5 [==============================] - 5s 975ms/step - loss: 2.2827 - accuracy: 0.1232
Epoch 8/500
5/5 [==============================] - 5s 993ms/step - loss: 2.2823 - accuracy: 0.0942
Epoch 9/500
5/5 [==============================] - 5s 978ms/step - loss: 2.2824 - accuracy: 0.1232
Epoch 10/500
5/5 [==============================] - 5s 996ms/step - loss: 2.2820 - accuracy: 0.1087
Epoch 11/500
5/5 [==================

5/5 [==============================] - 5s 956ms/step - loss: 2.0138 - accuracy: 0.5580
Epoch 84/500
5/5 [==============================] - 5s 957ms/step - loss: 2.0107 - accuracy: 0.5942
Epoch 85/500
5/5 [==============================] - 5s 951ms/step - loss: 2.0271 - accuracy: 0.4710
Epoch 86/500
5/5 [==============================] - 5s 948ms/step - loss: 2.0072 - accuracy: 0.5435
Epoch 87/500
5/5 [==============================] - 5s 999ms/step - loss: 1.9938 - accuracy: 0.5362
Epoch 88/500
5/5 [==============================] - 5s 992ms/step - loss: 2.0066 - accuracy: 0.5145
Epoch 89/500
5/5 [==============================] - 5s 980ms/step - loss: 1.9918 - accuracy: 0.5362
Epoch 90/500
5/5 [==============================] - 5s 991ms/step - loss: 1.9860 - accuracy: 0.5797
Epoch 91/500
5/5 [==============================] - 6s 1s/step - loss: 1.9853 - accuracy: 0.5290
Epoch 92/500
5/5 [==============================] - 5s 1s/step - loss: 1.9963 - accuracy: 0.5290
Epoch 93/500
5/5 [=

5/5 [==============================] - 5s 971ms/step - loss: 1.5613 - accuracy: 0.8333
Epoch 165/500
5/5 [==============================] - 5s 963ms/step - loss: 1.5633 - accuracy: 0.7971
Epoch 166/500
5/5 [==============================] - 5s 990ms/step - loss: 1.5644 - accuracy: 0.8406
Epoch 167/500
5/5 [==============================] - 5s 1s/step - loss: 1.5335 - accuracy: 0.8696
Epoch 168/500
5/5 [==============================] - 5s 979ms/step - loss: 1.5281 - accuracy: 0.8623
Epoch 169/500
5/5 [==============================] - 5s 974ms/step - loss: 1.5271 - accuracy: 0.8623
Epoch 170/500
5/5 [==============================] - 5s 978ms/step - loss: 1.5243 - accuracy: 0.8841
Epoch 171/500
5/5 [==============================] - 5s 963ms/step - loss: 1.5205 - accuracy: 0.8333
Epoch 172/500
5/5 [==============================] - 5s 959ms/step - loss: 1.4984 - accuracy: 0.8841
Epoch 173/500
5/5 [==============================] - 5s 957ms/step - loss: 1.4984 - accuracy: 0.8841
Epoch 1

5/5 [==============================] - 5s 955ms/step - loss: 1.0669 - accuracy: 0.9638
Epoch 246/500
5/5 [==============================] - 5s 957ms/step - loss: 1.0698 - accuracy: 0.9783
Epoch 247/500
5/5 [==============================] - 5s 952ms/step - loss: 1.0619 - accuracy: 0.9565
Epoch 248/500
5/5 [==============================] - 5s 964ms/step - loss: 1.0853 - accuracy: 0.9855
Epoch 249/500
5/5 [==============================] - 5s 977ms/step - loss: 1.0463 - accuracy: 0.9493
Epoch 250/500
5/5 [==============================] - 5s 963ms/step - loss: 1.0484 - accuracy: 0.9493
Epoch 251/500
5/5 [==============================] - 5s 951ms/step - loss: 1.0408 - accuracy: 0.9638
Epoch 252/500
5/5 [==============================] - 5s 973ms/step - loss: 1.0490 - accuracy: 0.9783
Epoch 253/500
5/5 [==============================] - 5s 965ms/step - loss: 1.0590 - accuracy: 0.9420
Epoch 254/500
5/5 [==============================] - 5s 955ms/step - loss: 1.0470 - accuracy: 0.9710
Epoc

5/5 [==============================] - 5s 967ms/step - loss: 0.6829 - accuracy: 0.9928
Epoch 327/500
5/5 [==============================] - 5s 998ms/step - loss: 0.6795 - accuracy: 1.0000
Epoch 328/500
5/5 [==============================] - 5s 982ms/step - loss: 0.6807 - accuracy: 0.9928
Epoch 329/500
5/5 [==============================] - 5s 975ms/step - loss: 0.6884 - accuracy: 1.0000
Epoch 330/500
5/5 [==============================] - 5s 1s/step - loss: 0.6632 - accuracy: 0.9928
Epoch 331/500
5/5 [==============================] - 5s 975ms/step - loss: 0.6694 - accuracy: 1.0000
Epoch 332/500
5/5 [==============================] - 5s 1s/step - loss: 0.6769 - accuracy: 1.0000
Epoch 333/500
5/5 [==============================] - 5s 980ms/step - loss: 0.6447 - accuracy: 1.0000
Epoch 334/500
5/5 [==============================] - 5s 981ms/step - loss: 0.6581 - accuracy: 1.0000
Epoch 335/500
5/5 [==============================] - 5s 982ms/step - loss: 0.6651 - accuracy: 0.9855
Epoch 336/

5/5 [==============================] - 5s 1s/step - loss: 0.4518 - accuracy: 1.0000
Epoch 408/500
5/5 [==============================] - 6s 1s/step - loss: 0.4271 - accuracy: 1.0000
Epoch 409/500
5/5 [==============================] - 5s 972ms/step - loss: 0.4123 - accuracy: 1.0000
Epoch 410/500
5/5 [==============================] - 5s 992ms/step - loss: 0.4269 - accuracy: 1.0000
Epoch 411/500
5/5 [==============================] - 5s 1s/step - loss: 0.4350 - accuracy: 1.0000
Epoch 412/500
5/5 [==============================] - 5s 988ms/step - loss: 0.4349 - accuracy: 1.0000
Epoch 413/500
5/5 [==============================] - 5s 1s/step - loss: 0.4037 - accuracy: 1.0000
Epoch 414/500
5/5 [==============================] - 5s 1s/step - loss: 0.4197 - accuracy: 0.9928
Epoch 415/500
5/5 [==============================] - 5s 967ms/step - loss: 0.4150 - accuracy: 1.0000
Epoch 416/500
5/5 [==============================] - 5s 976ms/step - loss: 0.3991 - accuracy: 1.0000
Epoch 417/500
5/5 [

5/5 [==============================] - 5s 952ms/step - loss: 0.2769 - accuracy: 1.0000
Epoch 489/500
5/5 [==============================] - 5s 956ms/step - loss: 0.2574 - accuracy: 1.0000
Epoch 490/500
5/5 [==============================] - 5s 952ms/step - loss: 0.2664 - accuracy: 1.0000
Epoch 491/500
5/5 [==============================] - 5s 955ms/step - loss: 0.2734 - accuracy: 1.0000
Epoch 492/500
5/5 [==============================] - 5s 960ms/step - loss: 0.2586 - accuracy: 1.0000
Epoch 493/500
5/5 [==============================] - 5s 958ms/step - loss: 0.2665 - accuracy: 1.0000
Epoch 494/500
5/5 [==============================] - 5s 956ms/step - loss: 0.2502 - accuracy: 1.0000
Epoch 495/500
5/5 [==============================] - 5s 984ms/step - loss: 0.2595 - accuracy: 1.0000
Epoch 496/500
5/5 [==============================] - 5s 960ms/step - loss: 0.2597 - accuracy: 1.0000
Epoch 497/500
5/5 [==============================] - 5s 952ms/step - loss: 0.2548 - accuracy: 1.0000
Epoc

In [11]:
# from keras.preprocessing.image import load_img

# image = load_img(r'C:\Users\chandru\Desktop\dataset_different\dataset\4\color_001.jpg', target_size=(img_size, img_size))
# img = np.array(image)
# img = img / 255.0
# img = img.reshape(1,img_size,img_size,3)
# label = model.predict(img)
# label

array([[0.01052759, 0.00350363, 0.02042241, 0.9155297 , 0.00945489,
        0.00722548, 0.01809417, 0.00339858, 0.00690038, 0.00494309]],
      dtype=float32)

In [12]:
from keras.models import load_model

model.save('my_model_final.h5')  # creates a HDF5 file 'my_model.h5'
del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
model = load_model('my_model_final.h5')

In [14]:
import numpy as np
import cv2
import glob
from keras.preprocessing.image import load_img
from keras.models import load_model

label_list = ['Stop Navigation',
              'Excuse me',
              'I am sorry',
              'Thanks you',
              'Good bye',
              'I love this game',
              'Nice to meet you',
              'You are welcome',
              'How are you',
              'Have a good time']


index = []

# image size
img_size = 224

# Load Saved Model
model = load_model('my_model_final.h5')


# img_size = 244
# image = load_img(r'C:\Users\chandru\Desktop\dataset_different\dataset\4\color_001.jpg', target_size=(img_size, img_size))
# img = np.array(image)
# img = img / 255.0
# img = img.reshape(1, img_size, img_size, 3)
# label = model.predict(img)
# print(label)


# Loop Through image folder
for img in glob.glob(r"C:\Users\chandru\Desktop\LipReading\dataset\\*.jpg"):
    image = load_img(img, target_size=(img_size, img_size))
    img = np.array(image)
    img = img / 255.0
    img = img.reshape(1, img_size, img_size, 3)
    label = model.predict(img)
    classes_x = np.argmax(label, axis=1)[0]
    index.append(classes_x)


class_ = max(index)
print("The Phase is: ", label_list[class_])


array([[1.1461634e-02, 2.0387659e-02, 5.8624048e-02, 8.3826530e-01,
        4.8099045e-04, 2.3413530e-02, 2.1524174e-02, 1.2807905e-03,
        5.1583429e-03, 1.9403493e-02]], dtype=float32)